In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
pd.set_option('display.max_rows', None)

In [2]:
from absbox import API
localAPI = API("https://absbox.org/api/dev",'english',check=False)
#localAPI = API("http://localhost:8081",'english',check=False)

✅Connected, local lib:0.20.2, server:0.21.11

In [146]:
from absbox.local.generic import Generic

test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":30
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"Amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ]
     ,"Accelerated":[
         ["accrueAndPayInt","acc01",["A1"]]
         ,["payPrinResidual","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]         
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,{"AfterCollect":{
        "lowerThanBndTrigger": 
         {"condition":[("bondBalance",),">"
                       ,("sum"
                         ,("accountBalance",)
                         ,("poolBalance",)
                         ,("factor"
                           ,("cumPoolCollection","Delinquencies")
                           ,0.5)
                        )]
          ,"effects":("newStatus","Accelerated")
          ,"status":False
          ,"curable":False}}}
    ,("PreClosing","Amortizing")
    )

In [147]:
r = localAPI.run(test01
               ,poolAssump = ("Pool",("Mortgage","Delinq"
                                      ,{"DelinqCDR":0.17715,"Lag":2,"DefaultPct":0.8}
                                      ,None
                                      ,{"Rate":0.7,"Lag":3}
                                      ,None)
                                       ,None
                                       ,None)
               ,runAssump = [("inspect"
                              ,("MonthEnd",("status",("Amortizing")))
                              ,("MonthEnd",("accountBalance",))
                              ,("MonthEnd",("poolBalance",))
                              ,("MonthEnd",("factor"
                                               ,("cumPoolCollection","Delinquencies")
                                               ,0.5))
                              ,("MonthEnd",("bondBalance",))
                              ,("MonthEnd",("trigger","AfterCollect","lowerThanBndTrigger"))
                             )]
               ,read=True)
#r['pool']['flow']

⠋

In [148]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Delinquency,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty
Date,,,,,,,,,,,
2021-03-01,2296.91,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,None,None
2021-06-15,1907.82,258.78,53.96,0,130.31,53.72,0.00,0.00,0.08,None,None
2021-07-31,1815.46,63.57,12.00,0,28.79,25.46,0.00,0.00,0.08,None,None
2021-08-31,1724.06,63.16,11.42,0,28.24,25.04,18.28,7.83,0.08,None,None
2021-09-30,1634.58,62.73,10.85,0,26.75,23.03,19.32,8.28,0.08,None,None
2021-10-31,1547.74,62.35,10.29,0,24.49,22.59,17.82,7.63,0.08,None,None
2021-11-30,1461.90,61.95,9.74,0,23.89,21.40,17.52,7.51,0.08,None,None
2021-12-31,1378.59,61.53,9.22,0,21.78,19.59,16.12,6.90,0.08,None,None
2022-01-31,1296.32,61.11,8.71,0,21.16,19.11,15.81,6.77,0.08,None,None


In [149]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-07-26,695.12,7.86,304.88,0.07,312.74,"[<PayInt:A1>, <PayPrin:A1>]"
2021-08-20,622.88,3.33,72.24,0.07,75.57,"[<PayInt:A1>, <PayPrin:A1>]"
2021-09-20,533.72,3.70,89.16,0.07,92.86,"[<PayInt:A1>, <PayPrin:A1>]"
2021-10-20,443.89,3.07,89.83,0.07,92.90,"[<PayInt:A1>, <PayPrin:A1>]"
2021-11-20,356.06,2.63,87.83,0.07,90.46,"[<PayInt:A1>, <PayPrin:A1>]"
2021-12-20,268.89,2.04,87.17,0.07,89.21,"[<PayInt:A1>, <PayPrin:A1>]"
2022-01-20,183.61,1.59,85.28,0.07,86.87,"[<PayInt:A1>, <PayPrin:A1>]"
2022-02-20,99.07,1.09,84.54,0.07,85.63,"[<PayInt:A1>, <PayPrin:A1>]"
2022-03-20,15.72,0.53,83.35,0.07,83.88,"[<PayInt:A1>, <PayPrin:A1>]"


In [150]:
r['result']['status']

,Date,From,To
0,2021-06-15,PreClosing,Amortizing
1,2021-06-15,Amortizing,Accelerated


In [136]:
from absbox.local.util import unifyTs

tieout = unifyTs(r['result']['inspect'].values())

In [151]:
tieout['left'] = tieout['<AccBalance:>']+tieout['<CurrentPoolBalance>']+tieout["<Factor:{'tag': 'PoolCumCollection', 'contents': ['NewDelinquencies']},{'numerator': 1, 'denominator': 2}>"]

In [138]:
tieout

,<AccBalance:>,<CurrentBondBalance>,<CurrentPoolBalance>,"<Factor:{'tag': 'PoolCumCollection', 'contents': ['NewDelinquencies']},{'numerator': 1, 'denominator': 2}>",<IsDealStatus>,"<TriggersStatus:EndCollectionWF,lowerThanBndTrigger>",left
Date,,,,,,,
2021-03-01,0,2000,0,0,False,False,0
2021-03-31,0,2000,0,0,False,False,0
2021-04-30,0,2000,0,0,False,False,0
2021-05-31,0,2000,0,0,False,False,0
2021-06-30,0,2000,2200,0,True,False,2200
2021-07-31,0,2000,2200,0,True,False,2200
2021-08-31,0,2000,2200,0,True,False,2200
2021-09-30,0,2000,2200,0,True,False,2200
2021-10-31,0,2000,2200,0,True,False,2200


In [152]:
fn = lambda xs: [ (ps,r) for (ps,r) in xs]

In [153]:
fn([(1,2),("A","B")])

[(1, 2), ('A', 'B')]